In [82]:
import pandas as pd
import scipy.stats as sts
from sklearn import preprocessing as prep
import numpy as np

df = pd.read_excel('cervejas.xlsx')

df

,Nome,Tipo,País,Teor Alcoólico,Volume Garrafa,Preço,Lixo
0,Baden Baden American IPA,IPA,Brasil,6.40,600,18.90,23
1,Baden Baden Chocolate,Specialty,Brasil,6.00,600,19.80,23
2,Brewmeister Armaggedon,Strong Golden Ale,Escócia,65.00,330,265.00,23
3,Crystal,Pilsner,Brasil,4.50,355,2.95,23
4,Delirium Tremens,Strong Golden Ale,Bélgica,8.50,330,29.67,23
5,Eisenbahn Pilsen,Pilsner,Brasil,4.80,355,4.90,23
6,Evil Twin Falco,IPA,Dinamarca,7.00,355,29.90,23
7,Golden Queen Bee,Strong Golden Ale,Bélgica,8.80,750,270.00,23
8,Kasteel Tripel,Trippel,Bélgica,11.00,330,32.90,23
9,Licher Weizen,German Weizen,Alemanha,5.40,500,21.90,23


In [83]:
df.drop([2, 7, 12], inplace=True)
df.drop(columns=['Lixo'], inplace=True)

numeric_data = df._get_numeric_data()
maxima = numeric_data.max()
minima = numeric_data.min()

data_range = maxima - minima
data_range[data_range == 0] = 1.0

numeric_data = (numeric_data - minima) / data_range

df[numeric_data.columns] = numeric_data

df


,Nome,Tipo,País,Teor Alcoólico,Volume Garrafa,Preço
0,Baden Baden American IPA,IPA,Brasil,0.292308,1.000000,0.532554
1,Baden Baden Chocolate,Specialty,Brasil,0.230769,1.000000,0.562604
3,Crystal,Pilsner,Brasil,0.000000,0.092593,0.000000
4,Delirium Tremens,Strong Golden Ale,Bélgica,0.615385,0.000000,0.892154
5,Eisenbahn Pilsen,Pilsner,Brasil,0.046154,0.092593,0.065109
6,Evil Twin Falco,IPA,Dinamarca,0.384615,0.092593,0.899833
8,Kasteel Tripel,Trippel,Bélgica,1.000000,0.000000,1.000000
9,Licher Weizen,German Weizen,Alemanha,0.138462,0.629630,0.632721
10,Lohn Bier Trippel,Trippel,Brasil,0.907692,0.000000,0.565943
11,Skol,Pilsner,Brasil,0.015385,0.166667,0.001336


In [84]:
le = prep.LabelEncoder()
stats = {}
for i,a in enumerate(df.columns):
    stats[a] = {}
    stats[a]['type'] = df.dtypes[i]
    ## Counting
    stats[a]['count'] = df[a].count()
    ## Non-unique values
    stats[a]['nunique'] = df[a].nunique()
    ## Mode
    mode = df[a].mode()
    if len(mode) == 1:
        stats[a]['mode'] = mode[0]
    else:
        stats[a]['mode'] = None
    
    if pd.api.types.is_numeric_dtype(df[a]):
        ## Entropy
        hist = np.histogram(df[a])[0]
        p = hist / np.sum(hist)
        stats[a]['entropy'] = sts.entropy(p)
        ## Variance
        stats[a]['variance'] = df[a].var()
        ## Average
        stats[a]['average'] = df[a].mean()
        ## Dispersion
        if stats[a]['average']:
            stats[a]['dispersion'] = stats[a]['variance']/stats[a]['average']
        else:
            stats[a]['dispersion'] = 0.0
        ## Standard deviation
        stats[a]['std_dev'] = df[a].std()
        ## Minimum and maximum
        stats[a]['min'] = df[a].min()
        stats[a]['max'] = df[a].max()
        ## Median
        stats[a]['median'] = df[a].median()
        ## Skewness and Kurtosis
        stats[a]['skewness'] = df[a].skew()
        stats[a]['kurtosis'] = df[a].kurt()
        ## Quantiles
        qts = df[a].quantile([0.25, 0.5, 0.75])
        stats[a]['quantile1'] = qts[0.25]
        stats[a]['quantile2'] = qts[0.5]
        stats[a]['quantile3'] = qts[0.75]
    else:
        tmp = le.fit_transform(df[a])
        hist = np.histogram(tmp)[0]
        p = hist / np.sum(hist)
        stats[a]['entropy'] = sts.entropy(p)
        stats[a]['variance'] = None
        stats[a]['average'] = None
        stats[a]['dispersion'] = None
        stats[a]['std_dev'] = None
        stats[a]['min'] = None
        stats[a]['max'] = None
        stats[a]['median'] = None
        stats[a]['skewness'] = None
        stats[a]['kurtosis'] = None
        stats[a]['quantile1'] = None
        stats[a]['quantile2'] = None
        stats[a]['quantile3'] = None
        
df_stats = pd.DataFrame.from_dict(stats, orient='index')

df_stats
        

,type,count,nunique,mode,entropy,variance,average,dispersion,std_dev,min,max,median,skewness,kurtosis,quantile1,quantile2,quantile3
Nome,object,10,10,None,2.302585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
País,object,10,4,Brasil,1.088900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Preço,float64,10,10,None,1.504788,0.141786,0.515225,0.275192,0.376545,0.0,1.0,0.564274,-0.357551,-1.312956,0.181970,0.564274,0.827295
Teor Alcoólico,float64,10,10,None,1.695743,0.132502,0.363077,0.364943,0.364009,0.0,1.0,0.261538,0.864627,-0.590587,0.069231,0.261538,0.557692
Tipo,object,10,6,Pilsner,1.695743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Volume Garrafa,float64,10,5,None,1.088900,0.167215,0.307407,0.543954,0.408920,0.0,1.0,0.092593,1.174506,-0.400974,0.023148,0.092593,0.513889


In [85]:
numeric_fields = df._get_numeric_data().columns

df_stats.loc[numeric_fields]


,type,count,nunique,mode,entropy,variance,average,dispersion,std_dev,min,max,median,skewness,kurtosis,quantile1,quantile2,quantile3
Teor Alcoólico,float64,10,10,None,1.695743,0.132502,0.363077,0.364943,0.364009,0.0,1.0,0.261538,0.864627,-0.590587,0.069231,0.261538,0.557692
Volume Garrafa,float64,10,5,None,1.088900,0.167215,0.307407,0.543954,0.408920,0.0,1.0,0.092593,1.174506,-0.400974,0.023148,0.092593,0.513889
Preço,float64,10,10,None,1.504788,0.141786,0.515225,0.275192,0.376545,0.0,1.0,0.564274,-0.357551,-1.312956,0.181970,0.564274,0.827295


In [86]:
from IPython.core.display import display, HTML
display(HTML('<h2 style="text-align:center">Stored Data Description</h2>' + df_stats.to_html()))

print('Stored Data Description\n' + str(df_stats))


,type,count,nunique,mode,entropy,variance,average,dispersion,std_dev,min,max,median,skewness,kurtosis,quantile1,quantile2,quantile3
Nome,object,10,10,None,2.302585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
País,object,10,4,Brasil,1.088900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Preço,float64,10,10,None,1.504788,0.141786,0.515225,0.275192,0.376545,0.0,1.0,0.564274,-0.357551,-1.312956,0.181970,0.564274,0.827295
Teor Alcoólico,float64,10,10,None,1.695743,0.132502,0.363077,0.364943,0.364009,0.0,1.0,0.261538,0.864627,-0.590587,0.069231,0.261538,0.557692
Tipo,object,10,6,Pilsner,1.695743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Volume Garrafa,float64,10,5,None,1.088900,0.167215,0.307407,0.543954,0.408920,0.0,1.0,0.092593,1.174506,-0.400974,0.023148,0.092593,0.513889


Stored Data Description
                   type  count  nunique     mode   entropy  variance  \
Nome             object     10       10     None  2.302585       NaN   
País             object     10        4   Brasil  1.088900       NaN   
Preço           float64     10       10     None  1.504788  0.141786   
Teor Alcoólico  float64     10       10     None  1.695743  0.132502   
Tipo             object     10        6  Pilsner  1.695743       NaN   
Volume Garrafa  float64     10        5     None  1.088900  0.167215   

                 average  dispersion   std_dev  min  max    median  skewness  \
Nome                 NaN         NaN       NaN  NaN  NaN       NaN       NaN   
País                 NaN         NaN       NaN  NaN  NaN       NaN       NaN   
Preço           0.515225    0.275192  0.376545  0.0  1.0  0.564274 -0.357551   
Teor Alcoólico  0.363077    0.364943  0.364009  0.0  1.0  0.261538  0.864627   
Tipo                 NaN         NaN       NaN  NaN  NaN       NaN     